In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df=pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
df.info()

In [ ]:
df.head()

In [ ]:
df=df.drop(["id"],axis="columns")

In [ ]:
from sklearn import preprocessing 
le=preprocessing.LabelEncoder() 
df["gender"]=le.fit_transform(df["gender"])
df["ever_married"]=le.fit_transform(df["ever_married"])
df["Residence_type"]=le.fit_transform(df["Residence_type"])
df["work_type"]=le.fit_transform(df["work_type"])
df["smoking_status"]=le.fit_transform(df["smoking_status"])
df.head()

In [ ]:
df = df.dropna(axis=0, subset=['bmi'])

In [ ]:
df.head()

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols_to_scale=["age","avg_glucose_level","bmi"]
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [ ]:
df.head()

In [ ]:
df["stroke"].value_counts()
#this data is totally imbalanced!!!
#lets create a new balanced database

In [ ]:
df_new1=df[df["stroke"]==0]
df_new2=df[df["stroke"]==1]

In [ ]:
df_new1.shape
df_new2.shape
df_new1_final=df_new1.sample(300)

In [ ]:
df_fotrain=pd.concat([df_new1_final,df_new2],axis=0)
df_fotrain.shape

In [ ]:
df_fotrain.head()

In [ ]:
inputs=df_fotrain.drop(["stroke"],axis="columns")
output=df_fotrain["stroke"]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(inputs,output, test_size=0.1, random_state=1)

In [ ]:
import tensorflow as tf
from tensorflow import keras
es=tf.keras.callbacks.EarlyStopping(
    min_delta=0.001,
    patience=10,
    restore_best_weights=True
)
model=keras.Sequential([
    keras.layers.Dense(100,input_shape=[10], activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(200, input_shape=[10], activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1,activation="sigmoid")
])
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(inputs,output,epochs=50,batch_size=4,callbacks=[es])

In [ ]:
y_pred=model.predict(x_test).flatten()
y_pred=np.round(y_pred)

In [ ]:
y_pred[:10]


In [ ]:
y_test[:10]
# all values matched in this small lot. Lol we got lucky i guess !!!
#Lets run out a classification report

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))
#hmmn the f1 score is pretty good. Also we have used a deep learning model on a small sample. So....:)

In [ ]:
cf=confusion_matrix(y_test,y_pred)
print(cf)
#Hmmn not interesting....lets make it visually appealing:)

In [ ]:
import seaborn as sns
sns.heatmap(cf, annot=True)
#so out of 51 times, we predicted 40 times right!